In [3]:
from langchain_community.llms import Ollama
from langchain_core.output_parsers import JsonOutputParser
from langchain.output_parsers import YamlOutputParser

In [4]:
import pandas as pd
import numpy as np

In [5]:
llm = Ollama(model="gemma:7b")

In [6]:
from langchain_core.prompts import ChatPromptTemplate
import re

def extract_cypher_scripts(text):
    cypher_list = list()
    for line in text.split("\n"):
        if 'CREATE' in line or 'MATCH' in line or 'CONNECT' in line:
            cypher_list.append(line)
            
    return cypher_list

In [7]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a ontologist. You build knowledge graph from plain input. Create Knowledge Graph in Cypher Script"),    
    ("user", "{input}")
])

chain = prompt | llm

In [8]:
contexts = pd.read_csv('/Users/mertakcay/Projects/KnowledgeGraphBuilder/context.csv')
cypher_list = list()
for context in contexts['context'].values:
    output = chain.invoke({"input": context})
    output = extract_cypher_scripts(output)
    print(output)
    cypher_list.append(output)
    print('*****************')
    # for sample in entities.split('\n'):
    #     if '-' in sample:
    #         entity_list.append(sample.replace('- ',''))

['CREATE ( artist:Person { name: "Beyoncé Giselle Knowles-Carter", born: 1981-09-04, nationality: "American", occupation: "Singer, Songwriter, Record Producer, Actress" } )', 'CREATE ( artist )-[:HAS_ORIGIN]->( place:Location { name: "Houston, Texas" } )', 'CREATE ( artist )-[:BECAME_KNOWN_FOR]->( achievement:Achievement { name: "Lead singer of R&B girl-group Destiny\'s Child", accomplishment: "Global fame and success" } )', 'CREATE ( artist )-[:RECORDED_ALBUM]->( album:Album { name: "Dangerously in Love", year: 2003, singles: ["Crazy in Love", "Baby Boy"] } )', 'CREATE ( album )-[:WON_AWARDS]->( award:Award { name: "Grammy Award", quantity: 5 } )']
*****************
['CREATE (b:Beyoncé)', "CREATE (sd:Destiny'sChild)", "CREATE (bd:B'Day {year: 2006})", 'CREATE (d:Déjà Vu)', 'CREATE (i:Irreplaceable)', 'CREATE (bl:BeautifulLiar)', 'CREATE (dg:Dreamgirls {year: 2006})', 'CREATE (tp:The Pink Panther {year: 2006})', 'CREATE (o:Obsessed {year: 2009})', 'CREATE (cr:Cadillac Records {year: 20